In [1]:
import requests
import re
from random import randint
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

In [2]:
#def data_film(genre='',sort='', rating='', subscript='available_online', country='', kind=''(film/serial), page=''):
def data_film(l):
    l = l.split('| ')
    url = f"https://www.kinopoisk.ru/lists/navigator/{l[0]}/?sort={l[1]}&quick_filters=%2C{l[2]}%2C{l[3]}%2C{l[4]}%2C{l[5]}&tab=online&page={l[6]}"
    dr = webdriver.Chrome()
    try:
        dr.get(url)
    except:
        return ([])
    print(url)
    soup = BeautifulSoup(dr.page_source)
    res = soup.find_all('div', {'class' :'desktop-seo-selection-film-item__upper-wrapper'})
    return (res)

In [3]:
def get_row(i, raw_data):
    name = raw_data[i].find_all('p', {'class' : 'selection-film-item-meta__name'}, limit = 1)
    if (len(name) > 0): name = name[0].text 
    else: return 0
    
    link = raw_data[i].find_all('a', {'class' : 'selection-film-item-meta__link'}, limit = 1)
    if (len(link) > 0): link = 'https://www.kinopoisk.ru' + link[0].get('href') 
    else: return 0
    
    date = raw_data[i].find_all('p', {'class' : 'selection-film-item-meta__original-name'}, limit = 1)
    if (len(date) > 0): date = date[0].text 
    else: return 0
    
    date = re.search(r'[\d]{4}$', date)
    if (date != None): date = date.group(0) 
    else: return 0
    
    x = raw_data[i].find_all('span', {'class' : 'selection-film-item-meta__meta-additional-item'}, limit = 2)
    if (len(x) > 1): country = x[0].text; gen = x[1].text 
    else: return 0
    
    rating = raw_data[i].find_all('span', {'class' : r'rating__value rating__value_positive'}, limit = 1)
    if (len(rating) == 0): rating = raw_data[i].find_all('span', {'class' : r'rating__value rating__value_neutral'}, limit = 1)
    if (len(rating) > 0): rating = rating[0].text 
    else: return 0
    
    rating_count = raw_data[i].find_all('span', {'class' : 'rating__count'}, limit = 1)
    if (len(rating_count) > 0): rating_count = rating_count[0].text 
    else: return 0
    
    return [name, date, country, gen, rating, rating_count, link]

In [114]:
def get_add_info(df, name):
    df = df.set_index('name')
    elem = df.loc[name]
    link = str(elem['link'])
    dr = webdriver.Chrome()
    try:
        dr.get(link)
    except:
        return([])
    soup1 = BeautifulSoup(dr.page_source)
    info = soup1.find_all('div', {'class' : 'styles_valueLight__1j0RO'}, limit = 2)
    if (len(info) > 1): audio_track = info[0].text; subtitles = info[1].text
    else: return 0
    
    description = soup1.find_all('p', {'class' : 'styles_paragraph__2Otvx'}, limit = 1)
    if (len(description) > 0): description = description[0].text
    else: return 0
    country = str(elem['country'])
    date = str(elem['date'])
    s = 'Страна:\\n' + country + '\\n\\n'
    s += 'Год:\\n' + date + '\\n\\n'
    s += 'Язык озвучки:\\n' + audio_track + '\\n\\n'
    s += 'Субтитры:\\n' + subtitles + '\\n\\n'
    s += 'Описание:\\n' + description
    return s

In [115]:
s = get_add_info(df, 'Ford против Ferrari')
print(s)

Страна:\nСША\n\nГод:\n2019\n\nЯзык озвучки:\nРусский, Английский\n\nСубтитры:\nРусские, Английские\n\nОписание:\nВ начале 1960-х Генри Форд II принимает решение улучшить имидж компании и сменить курс на производство более модных автомобилей. После неудавшейся попытки купить практически банкрота Ferrari американцы решают бросить вызов итальянским конкурентам на трассе и выиграть престижную гонку 24 часа Ле-Мана. Чтобы создать подходящую машину, компания нанимает автоконструктора Кэррола Шэлби, а тот отказывается работать без выдающегося, но, как считается, трудного в общении гонщика Кена Майлза. Вместе они принимаются за разработку впоследствии знаменитого спорткара Ford GT40.


In [90]:
def get_ten(num, l):
    l += str((num - 1) // 50 + 1)
    num = (num - 1) % 50 
    raw_data = data_film(l)
    if (len(raw_data) == 0):
        return (0)
    df = pd.DataFrame(columns=['name','date','country','genre','rating','rating_count', 'link', 'audio', 'subtitles', 'description'])
    max_ind = len(raw_data)
    count = 0
    while (num < max_ind and count < 10):
        a = get_row(num , raw_data)
        if (a):
            a.extend([0, 0, 0])
            df.loc[count] = a
            count += 1
        num += 1
    return df

In [62]:
#data_film(genre='',sort='', rating='', subscript='available_online', country='', kind=''(film/serial), page=''):
genre = ['', 'anime', 'mystery', 'comedy', 'crime', 'adventure', 'family', 'thriller', 'horror', 'sci-fi', 'fantasy']
sort = ['', 'popularity', 'year']
rating = ['', 'high_rated']
subscript= ['available_online', 'yandex_plus_subscription', 'kinopoisk_with_amediateka_subscription']
country = ['', 'foreign', 'russian']
kind = ['', 'films', 'serials']

In [91]:
#sort=year&quick_filters=films%2Cforeign%2Cyandex_plus_subscription
search_param = '| | | yandex_plus_subscription| foreign| | '
#{'genre': '', 'sort': 'year', 'rating': '', 'subscript': 'yandex_plus_subscription',\
 #               'country': 'foreign', 'kind': 'films', 'page': ''}
#data_film(genre='',sort='', rating='', subscript='available_online', country='', kind=''(film/serial), page=''):
df = get_ten(51, search_param)

https://www.kinopoisk.ru/lists/navigator//?sort=&quick_filters=%2C%2Cyandex_plus_subscription%2Cforeign%2C&tab=online&page=2


In [92]:
def df_to_str(df):
    df = df.set_index('name')
    s = ''
    for i in range(len(df)):
        name = df.index[i]
        s += '<b>' + str(name) + '</b>\\n\\n'
        s += 'Рейтинг' + ' ' * 17 + '-' + ' ' * 5 + str(df.loc[name]['rating']) + '\\n'
        s += 'Кол-во оценок     ' + '-' + ' ' * 5 + str(df.loc[name]['rating_count']) + '\\n'
        genre_list = str(df.loc[name]['genre']).split(', ')
        s += 'Жанр' + ' ' * 22 + '-' + ' ' * 5 + genre_list[0]
        if len(genre_list) == 2:
            s += '\\n' + ' ' * 39 + genre_list[1]
        s += '\\n'
        if i != len(df) - 1:
            s += '\\n\\n\\n'
    return(s)
s = df_to_str(df)
print(s)

<b>Ford против Ferrari</b>\n\nРейтинг                 -     8.2\nКол-во оценок     -     398 140\nЖанр                      -     биография\n                                       спорт\n\n\n\n<b>Легенда</b>\n\nРейтинг                 -     7.2\nКол-во оценок     -     387 786\nЖанр                      -     криминал\n                                       триллер\n\n\n\n<b>Алита: Боевой ангел</b>\n\nРейтинг                 -     7.1\nКол-во оценок     -     384 731\nЖанр                      -     фантастика\n                                       боевик\n\n\n\n<b>Довод</b>\n\nРейтинг                 -     7.6\nКол-во оценок     -     384 056\nЖанр                      -     фантастика\n                                       боевик\n\n\n\n<b>Пираты Карибского моря: На краю света</b>\n\nРейтинг                 -     8.0\nКол-во оценок     -     379 535\nЖанр                      -     приключения\n                                       боевик\n\n\n\n<b>Дэдпул 2</b>\n\nРейтинг         

In [134]:
df.to_csv('csv_test.csv', index=False)
df1 = pd.read_csv("csv_test.csv")
#s = df_to_str(df1)
#print(df1)
#df1 = df1.set_index('name')
index = df1[df1['name'].isin(['Ford против Ferrari', 'Великий Гэтсби']) == False]
#df1 = df1.drop(index)
# print('\n\n')
print(index)

                                    name  date                  country  \
1                                Легенда  2015  Великобритания, Франция   
2                    Алита: Боевой ангел  2019              США, Япония   
3                                  Довод  2020      Великобритания, США   
4  Пираты Карибского моря: На краю света  2007                      США   
5                               Дэдпул 2  2018              США, Канада   
6                     Богемская рапсодия  2018      Великобритания, США   
7                    Корпорация монстров  2001                      США   
8                            Доктор Хаус  2012                      США   

                 genre  rating rating_count  \
1    криминал, триллер     7.2      387 786   
2   фантастика, боевик     7.1      384 731   
3   фантастика, боевик     7.6      384 056   
4  приключения, боевик     8.0      379 535   
5   фантастика, боевик     7.4      376 298   
6    биография, музыка     8.0      374 62